This project utilizes the PokeAPI (https://pokeapi.co/) to create a large database containing relvent information on Pokemon to be used for future projects. Note: This is not fully clean data. Working to wrangle further

Import the required Libraries

In [1]:
import requests
import pandas as pd

Create The Inital pd dataframes

In [2]:
pokedf1 = pokedf =  pd.DataFrame(columns = ['name','species', 'height', 'weight', 'abilities', 'types', 'stats', 'hp', 'attack', 'defence', 'sp_attack', 'sp_defence', 'speed'])
specdf1 = specdf = pd.DataFrame(columns = ['name','pokedex','gen','genus', 'capture_rate', 'color', 'shape', 'egg_groups', 'is_baby', 'is_legendary', 'is_mythical'])

Functions to update the dataframes

In [3]:
def update_pokedf(pokemon_info):
    #appending rows
    new_row = pokemon_info
    pokedf1.loc[len(pokedf1)] = new_row

    return None

def update_specdf(species_info):

    new_row = species_info
    specdf1.loc[len(specdf1)] = new_row

    return None

Get the pokemon info and append to df

In [4]:
def get_pokemon_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        pokemon_data = response.json()
        pokemon_info = {
            "name": pokemon_data["name"],
            "species": pokemon_data["species"]["name"],
            "height": pokemon_data["height"],
            "weight": pokemon_data["weight"],
            "abilities": [ability["ability"]["name"] for ability in pokemon_data["abilities"]],
            "types": [stats_data["type"]["name"] for stats_data in pokemon_data["types"]],
            "stats": [stats_data["base_stat"] for stats_data in pokemon_data["stats"]],
            "hp": pokemon_data["stats"][0]["base_stat"],
            "attack": pokemon_data["stats"][1]["base_stat"],
            "defence": pokemon_data["stats"][2]["base_stat"],
            "sp_attack": pokemon_data["stats"][3]["base_stat"],
            "sp_defence": pokemon_data["stats"][4]["base_stat"],
            "speed": pokemon_data["stats"][5]["base_stat"],
        }
        update_pokedf(pokemon_info)
        return pokemon_info
    else:
        return None

Get the species info and append to df

In [5]:
def get_species_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        species_data = response.json()
        species_info = {
            "name": species_data["name"],
            "pokedex": species_data["pokedex_numbers"][0]["entry_number"],
            "gen": species_data["generation"]["name"],
            #"genus": species_data["genera"]["genus"],
            "capture_rate": species_data["capture_rate"],
            "color": species_data["color"]["name"],
            #"shape": species_data["shape"]["name"],
            "egg_groups": [group["name"] for group in species_data["egg_groups"]],
            "is_baby": species_data["is_baby"],
            "is_legendary": species_data["is_legendary"],
            "is_mythical": species_data["is_mythical"],            
        }
        update_specdf(species_info)
        return species_info
    else:
        return None

Create the Pokemon df

In [6]:
url = f"https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"    #Real URL
#url = f"https://pokeapi.co/api/v2/pokemon?limit=100&offset=0"        #test URL
response = requests.get(url)

poke = response.json()

for pokemon in poke['results']:
    get_pokemon_info(pokemon['url'])

Create the Species df

In [7]:
url = f"https://pokeapi.co/api/v2/pokemon-species?limit=100000&offset=0"   #Real URL
#url = f"https://pokeapi.co/api/v2/pokemon-species?limit=100&offset=0"    #test URL
response = requests.get(url)

species = response.json()

for spec in species['results']:
    get_species_info(spec['url'])

#print(specdf1)

Function to edit generation format in data

In [8]:
def get_generation(generation):
    if generation == 'generation-i':
        return 1
    elif generation == 'generation-ii':
        return 2
    elif generation == 'generation-iii':
        return 3
    elif generation == 'generation-iv':
        return 4
    elif generation == 'generation-v':
        return 5
    elif generation == 'generation-vi':
        return 6
    elif generation == 'generation-vii':
        return 7
    elif generation == 'generation-viii':
        return 8
    elif generation == 'generation-ix':
        return 9
    else:
        return "NA"

Format Generation to numeric --- 
Join Pokemon/Species df --- 
Export to CSV

In [9]:
specdf1["generation"] = specdf1["gen"].apply(get_generation)
specdf1 = specdf1.drop("gen", axis = 1)

In [ ]:
df2 = pd.merge(pokedf1, specdf1, how="left", left_on='species', right_on='name')
df2 = df2.drop("name_y", axis = 1)

In [12]:
df2 = df2.sort_values(by=['pokedex', 'name_x'])
df2.to_csv("pokemon_data.csv", index = False)

Preview dataframes

In [22]:
pokedf1.head()

,name,species,height,weight,abilities,types,stats,hp,attack,defence,sp_attack,sp_defence,speed
0,bulbasaur,bulbasaur,7,69,"[overgrow, chlorophyll]","[grass, poison]","[45, 49, 49, 65, 65, 45]",45,49,49,65,65,45
1,ivysaur,ivysaur,10,130,"[overgrow, chlorophyll]","[grass, poison]","[60, 62, 63, 80, 80, 60]",60,62,63,80,80,60
2,venusaur,venusaur,20,1000,"[overgrow, chlorophyll]","[grass, poison]","[80, 82, 83, 100, 100, 80]",80,82,83,100,100,80
3,charmander,charmander,6,85,"[blaze, solar-power]",[fire],"[39, 52, 43, 60, 50, 65]",39,52,43,60,50,65
4,charmeleon,charmeleon,11,190,"[blaze, solar-power]",[fire],"[58, 64, 58, 80, 65, 80]",58,64,58,80,65,80


In [21]:
specdf1.head()

,name,pokedex,genus,capture_rate,color,shape,egg_groups,is_baby,is_legendary,is_mythical,generation
0,bulbasaur,1,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
1,ivysaur,2,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
2,venusaur,3,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
3,charmander,4,NaN,45,red,NaN,"[monster, dragon]",False,False,False,1
4,charmeleon,5,NaN,45,red,NaN,"[monster, dragon]",False,False,False,1


In [19]:
df2.head()

,name_x,species,height,weight,abilities,types,stats,hp,attack,defence,...,pokedex,genus,capture_rate,color,shape,egg_groups,is_baby,is_legendary,is_mythical,generation
0,bulbasaur,bulbasaur,7,69,"[overgrow, chlorophyll]","[grass, poison]","[45, 49, 49, 65, 65, 45]",45,49,49,...,1,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
1,ivysaur,ivysaur,10,130,"[overgrow, chlorophyll]","[grass, poison]","[60, 62, 63, 80, 80, 60]",60,62,63,...,2,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
2,venusaur,venusaur,20,1000,"[overgrow, chlorophyll]","[grass, poison]","[80, 82, 83, 100, 100, 80]",80,82,83,...,3,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
1219,venusaur-gmax,venusaur,240,10000,"[overgrow, chlorophyll]","[grass, poison]","[80, 82, 83, 100, 100, 80]",80,82,83,...,3,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
1057,venusaur-mega,venusaur,24,1555,[thick-fat],"[grass, poison]","[80, 100, 123, 122, 120, 80]",80,100,123,...,3,NaN,45,green,NaN,"[monster, plant]",False,False,False,1
